In [1]:
!pip install -qU textgrad
!pip install -qU together
!pip install -qU langchain
!pip install -qU langchain-community
!pip install -qU langchain-groq

In [1]:
import numpy as np
import random
import concurrent
import argparse
from dotenv import load_dotenv
from tqdm import tqdm
import textgrad as tg
from textgrad.tasks import load_task
load_dotenv(override = True)
import os

tg.set_backward_engine("ollama-gemma2", override = True)


def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)

In [54]:
import pandas as pd

df = pd.read_csv("/root/source/predict_hscode/before_update/code/tune prompt datasets/train.csv")
df2 = pd.read_csv("/root/source/predict_hscode/before_update/code/tune prompt datasets/valid.csv")
df3 = pd.read_csv("/root/source/predict_hscode/before_update/code/tune prompt datasets/test.csv")

df = df.dropna()
df2 = df2.dropna()
df3 = df3.dropna()

In [56]:
len(df3['hs_code'].unique())

6835

In [3]:
description = df['full_description'].to_list()
hs_code = df['hs_code'].to_list()

In [4]:
df_ = df[:100]
df2 = df2[:100]
df3 = df3[:100]

In [5]:
items_train = list(df_.itertuples(index = False,name = None))
items_valid = list(df2.itertuples(index = False,name = None))
items_test = list(df3.itertuples(index = False,name = None))

In [7]:
df4 = pd.read_csv("/root/source/predict_hscode/before_update/code/data_train.csv",dtype= {"hs_code":"object"})
df4.head()

,hs_code,description
0,010111,250 sacos 69 soppexcca product jinotega nicara...
1,010111,275 saco 69 soppexcca product jinote nicaragua...
2,010111,275 sacos 69 soppexcca product jinot ega nicar...
3,010111,275 sacos 69 soppexcca product jinot ega nicar...
4,010111,275 sacos 69 soppexcca product jinot ega nicar...


In [8]:
len(df4)

2501736

In [9]:
df4.isnull().sum()

hs_code          0
description    266
dtype: int64

In [10]:
df4 = df4.dropna()

In [11]:
len(df4)

2501470

In [12]:
len(df4['hs_code'].unique())

6835

In [13]:
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

documents = []

for index, row in df4.iterrows():
    text = row['description']
    hs_code = row['hs_code']
    documents.append(Document(page_content=text, metadata={'hs_code': hs_code}))

splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,  
    chunk_overlap=0
)

split_documents = []
for doc in documents:
    chunks = splitter.split_text(doc.page_content)
    for chunk in chunks:
        split_documents.append(Document(page_content=chunk, metadata=doc.metadata))

In [14]:
docs = []
for doc in split_documents:
    metadata = doc.metadata
    metadata_str = str(metadata).strip('{}')
    page = doc.page_content
    docs.append([metadata_str + " " + page])

In [15]:
cleaned_list = [item.replace('"', '').replace("'",'')  for items in docs for item in items]

In [16]:
import bm25s

# Create the BM25 model and index the corpus
retriever = bm25s.BM25(corpus=cleaned_list)
retriever.index(bm25s.tokenize(cleaned_list))

Split strings:   0%|          | 0/3753656 [00:00<?, ?it/s]

BM25S Count Tokens:   0%|          | 0/3753656 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/3753656 [00:00<?, ?it/s]

In [17]:
from langchain_core.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain_ollama import OllamaLLM
from langchain_groq import ChatGroq
from langchain.docstore.document import Document
import torch
import logging


template = """
    You are an expert in HS Code classification. 
    Based on the provided product description, accurately determine and return only one 6-digit HS Code that best matches the description.
    Only return the HS Code as a 6-digit number .
    Example:123456.
    Context:\n {context} \n
    Description:\n {description} \n
    Answer:

 """
 

prompt = PromptTemplate(template=template, input_variables=['context','description'])
llm = ""
device = "cuda" if torch.cuda.is_available() else "cpu"

try:
    llm = OllamaLLM(model="gemma2", temperature=0, device=device)
    
except Exception as e:
    logging.error(f"Failed to load model on {device}: {e}")
    llm = OllamaLLM(model="gemma2", temperature=0, device="cpu") 

chain = load_qa_chain(llm, chain_type = 'stuff', prompt = prompt)


def retrieval(sentence):
    sentence = sentence.lower()
    doc,_ = retriever.retrieve(bm25s.tokenize(sentence), k=5)
    context = [Document(d) for d in doc[0]] 
    return context

def predict(sentence,context,chain):
    response = chain.invoke({'input_documents': context, 'description':sentence})
    hscode= response.get("output_text")
    return hscode

/tmp/ipykernel_84328/1902215024.py:33: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain = load_qa_chain(llm, chain_type = 'stuff', prompt = prompt)


In [18]:
import re

def eval_sample(item, eval_fn,chain):
    y,x = item
    context = retrieval(x)
    response = predict(x,context,chain)
    response = response.replace("\n","").replace(" ","")
    result =""
    if len(response)>6:
        numbers = re.findall(r'\d+', response)[-1]

        numbers = numbers.replace("\n","").replace(" ","").replace('.','')

        result = tg.Variable(int(numbers),requires_grad = False, role_description = "predict value" )
    else:
        response = response.replace("\n","").replace(" ","")
        result = tg.Variable(response,requires_grad = False, role_description = "predict value" )

    y = tg.Variable(int(y), requires_grad = False, role_description = "correct answer for the query")
    
    try:
        eval_output_variable = eval_fn(inputs=dict(prediction=result, ground_truth_answer = y))
        return int(eval_output_variable.value)
    except:
        eval_output_variable =  eval_fn([x, y, result])
        eval_output_parsed = eval_fn.parse_output(eval_output_variable)
        return int(eval_output_parsed)

In [19]:
def eval_dataset(test_set, eval_fn, chain, max_samples: int=None):
    accuracy_list = []
    for sample in test_set:
        acc_item = eval_sample(sample, eval_fn,chain)
       # print("accuracy:",acc_item)
        accuracy_list.append(acc_item)
    print("accuracy:",np.mean(accuracy_list))
    return accuracy_list

In [20]:
import textgrad as tg

def run_validation_revert(system_prompt: tg.Variable, results, chain, eval_fn, val_set):
    val_performance = np.mean(eval_dataset(val_set, eval_fn, chain))
    previous_performance = np.mean(results["validation_acc"][-1])
    print("val_performance:", val_performance)
    print("previous_performance:", previous_performance)
    previous_prompt = results["prompt"][-1]
    
    if val_performance < previous_performance:
        print(f"rejected prompt: {system_prompt.value}")
        system_prompt.set_value(previous_prompt)
        val_performance = previous_performance
    results["validation_acc"].append(val_performance)

In [21]:
from textgrad.tasks import load_task

set_seed(12)
llm_api_eval = tg.get_engine("ollama-mistral")
llm_api_test = tg.get_engine("ollama-gemma2")
tg.set_backward_engine(llm_api_eval, override = True)

#load data and validation func
_, _, _, eval_fn = load_task("BBH_object_counting", evaluation_api=llm_api_eval)
train_set, val_set, test_set = items_train, items_valid, items_test
print("Train/Val/Test set lengths:", len(train_set), len(val_set),len(test_set))


Train/Val/Test set lengths: 100 100 100


In [22]:
import time
import numpy as np

train_loader = tg.tasks.DataLoader(train_set, batch_size=3, shuffle=True)

# Testing the 0-shot performance of the evaluation engine

system_prompt = tg.Variable("""
                You are an expert in HS Code classification. 
                Based on the provided product description, accurately determine and return only one 6-digit HS Code that best matches the description.
                Only return the HS Code as a 6-digit number .
                Example:123456
                Context:\n {context} \n
                Description:\n {description} \n
                Answer:
                """,
                requires_grad=True,
                role_description="Optimizes the provided system prompt to ensure that the generates and returns a single 6-digit HS Code accurately, based on the given product description.")

#model_evaluation = tg.BlackboxLLM(llm_api_eval, system_prompt)
optimizer = tg.TextualGradientDescent(engine=llm_api_eval,parameters=[system_prompt])

prompt_template = PromptTemplate(
    template=system_prompt.value,
    input_variables=["context","description"]
    )
chain = load_qa_chain(llm, chain_type = 'stuff', prompt = prompt_template)


results = {"test_acc": [], "prompt": [], "validation_acc": []}
results["test_acc"].append(eval_dataset(test_set, eval_fn, chain))

print("validation_acc")
results["validation_acc"].append(eval_dataset(val_set, eval_fn, chain))

print("prompt....")
results["prompt"].append(system_prompt.get_value())

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

...

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

accuracy: 0.73
prompt....


In [23]:
from tqdm import tqdm

for epoch in range(3):
    for steps, batch in enumerate((pbar := tqdm(train_loader))):
        batch_x,batch_y = batch[0], batch[1]
        pbar.set_description(f"Training step {steps}. Epoch {epoch}")
        optimizer.zero_grad()
        losses = []
        for (y, x) in zip(batch_x, batch_y):
            context = retrieval(x)
            response = predict(x,context,chain)
            reponse = response.replace("\n","").replace(" ","").replace('.','')
            result = tg.Variable(int(response),requires_grad = False, role_description = "predict value" )
            y = tg.Variable(int(y), requires_grad=False, role_description="correct answer for the query")
            try:
                eval_output_variable = eval_fn(inputs=dict(prediction=result, ground_truth_answer=y))
            except:
                eval_output_variable = eval_fn([x, y, result])

            losses.append(eval_output_variable)
 
        total_loss = tg.sum(losses)
        total_loss.backward()
        optimizer.step()
        
        run_validation_revert(system_prompt, results, chain, eval_fn, val_set)
        
        print("sys prompt: ", system_prompt)

        test_acc = eval_dataset(test_set, eval_fn, chain)
        results["test_acc"].append(test_acc)
        results["prompt"].append(system_prompt.get_value())
        if steps == 3:
            break                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

Training step 0. Epoch 0: : 0it [00:00, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

...

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

Training step 3. Epoch 2: : 3it [04:08, 82.71s/it]

accuracy: 0.74


In [25]:
len(results)

3

In [26]:
n = len(results["test_acc"])
test_acc = []
for i in range(n):
    acc = np.mean(results["test_acc"][i])
    test_acc.append(acc)
test_acc

[0.74, 0.74, 0.74, 0.74, 0.74, 0.74, 0.74, 0.74, 0.74, 0.74, 0.74, 0.74, 0.74]

In [27]:
pd.DataFrame({"prompt": results["prompt"],"test_acc": test_acc})

,prompt,test_acc
0,\n You are an expert in HS Code...,0.74
1,Provide a clear and concise system prompt that...,0.74
2,Request a concise and accurate 6-digit HS Code...,0.74
3,Provide a concise and accurate 6-digit HS Code...,0.74
4,Generate a precise 6-digit HS Code for the giv...,0.74
5,Identify and accurately provide a single 6-dig...,0.74
6,Determine and precisely provide a unique 6-dig...,0.74
7,Identify and precisely provide a 6-digit HS Co...,0.74
8,Provide a precise 6-digit HS Code for the give...,0.74
9,Generate a precise 6-digit HS Code for the spe...,0.74


In [28]:
pd.DataFrame({"prompt": results["prompt"],"test_acc": test_acc}).to_csv("/root/source/predict_hscode/before_update/code/result_prompt.csv")